In [74]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import cv2
import numpy as np
import os

train_folder = r"C:\Users\Adel\Downloads\fruits-360-100x100-main\Training"
test_folder = r"C:\Users\Adel\Downloads\fruits-360-100x100-main\Test"


ModuleNotFoundError: No module named 'tensorflow'

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras


ModuleNotFoundError: No module named 'pandas'

In [4]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


2.6.0+cu124
True
NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

classes = list(train_generator.class_indices.keys())
num_classes = len(classes)

with open("classes.txt", "w", encoding="utf-8") as f:
    for c in classes:
        f.write(c + "\n")

print("Number of classes:", num_classes)
print(classes[:15])


Found 119252 images belonging to 227 classes.
Found 39768 images belonging to 227 classes.
Number of classes: 227
['Apple 10', 'Apple 11', 'Apple 12', 'Apple 13', 'Apple 14', 'Apple 17', 'Apple 18', 'Apple 19', 'Apple 5', 'Apple 6', 'Apple 7', 'Apple 8', 'Apple 9', 'Apple Braeburn 1', 'Apple Core 1']


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # لتثبيت الطبقات الأساسية

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1
)

model.save("fruit_classifier_tl.h5")
print("Model Saved Successfully!")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,451,235 (9.35 MB)

 Trainable params: 193,251 (754.89 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
   5/3727 ━━━━━━━━━━━━━━━━━━━━ 40:16 649ms/step - accuracy: 0.0000e+00 - loss: 5.8761

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(test_generator)
print("Test Accuracy:", acc)
 
def predict_fruit(image_path):
    model = load_model("fruit_classifier_tl.h5")

    with open("classes.txt", "r", encoding='utf-8') as f:
        classes = [line.strip() for line in f.readlines()]

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("الصورة غير موجودة")

    original = img.copy()
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    prediction = model.predict(img, verbose=0)
    
    class_index = np.argmax(prediction)
    confidence = np.max(prediction)

    print(f"Prediction: {classes[class_index]}")
    print(f"Confidence: {confidence:.2%}")

    return classes[class_index]



In [ ]:
import torch

# نسخة PyTorch
print("PyTorch version:", torch.__version__)

# هل في GPU متاحة؟
print("Is CUDA available:", torch.cuda.is_available())

# لو في GPU اعرض اسمه
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("PyTorch running on CPU")


ModuleNotFoundError: No module named 'torch'

In [ ]:
predict_fruit(r"C:\Users\Adel\Downloads\pk.jpg")

In [7]:
import numpy as np

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = a + b
print("NumPy Test:", c)


NumPy Test: [5 7 9]


In [8]:
import pandas as pd

data = {'Name': ['Alice', 'Bob'], 'Age': [25, 30]}
df = pd.DataFrame(data)
print("Pandas Test:\n", df)


ModuleNotFoundError: No module named 'pandas'

In [9]:
import matplotlib.pyplot as plt

plt.plot([1, 2, 3], [4, 5, 6])
plt.title("Matplotlib Test")
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [10]:
import seaborn as sns
import pandas as pd

df = pd.DataFrame({'x':[1,2,3,4], 'y':[4,3,2,1]})
sns.lineplot(data=df, x='x', y='y')


ModuleNotFoundError: No module named 'seaborn'

In [11]:
from sklearn.linear_model import LinearRegression
import numpy as np

X = np.array([[1], [2], [3]])
y = np.array([2, 4, 6])

model = LinearRegression()
model.fit(X, y)
print("Scikit-learn Test Prediction for 4:", model.predict([[4]]))


ModuleNotFoundError: No module named 'sklearn'

In [12]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
a = tf.constant([[1, 2], [3, 4]])
b = tf.constant([[1, 1], [1, 1]])
print("TensorFlow Test:", tf.add(a, b))


ModuleNotFoundError: No module named 'tensorflow'

In [13]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(2, input_shape=(3,), activation='relu'),
    layers.Dense(1)
])
model.summary()


ModuleNotFoundError: No module named 'tensorflow'

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import tensorflow as tf

print("Pandas:", pd.__version__)
print("Matplotlib:", plt.__version__)
print("Seaborn:", sns.__version__)
print("Scikit-learn:", LinearRegression.__module__)
print("TensorFlow:", tf.__version__)


ModuleNotFoundError: No module named 'pandas'

In [4]:
import sys
print(sys.executable)  # هيظهر مسار البايثون اللي النوتبوك شغال عليه

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

print("All libraries are working!")


c:\Users\Adel\AppData\Local\Programs\Python\Python310\python.exe


ModuleNotFoundError: No module named 'pandas'

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("All good!")


ModuleNotFoundError: No module named 'pandas'